# Study of urban green space in relation to happiness

## Introduction - Problem Description

### Assisting the urban planning department of Brussels, Belgium to explore the density of urban green space and location for new green space.

A study of the University of Vermont shows a clear relationship between urban green space and happiness. The study was conducted using sentiment analysis (Hedonometer) on Twitter posts.
This allowed measurement of the level of happiness or positiveness by comparing the sentiment at the time before visiting, during visiting and after visiting urban parks.


Abstract from the study:
1. With more people living in cities, we are witnessing a decline in exposure to nature. A growing body of research has demonstrated an association between nature contact and improved mood.
2. Here, we used Twitter and the Hedonometer, a world analysis tool, to investigate how sentiment, or the estimated happiness of the words people write, varied before, during, and after visits to San Francisco's urban park system. We found that sentiment was substantially higher during park visits and remained elevated for several hours following the visit.
3. Leveraging differences in vegetative cover across park types, we explored how different types of outdoor public spaces may contribute to subjective well‐being. Tweets during visits to Regional Parks, which are greener and have greater vegetative cover, exhibited larger increases in sentiment than tweets during visits to Civic Plazas and Squares.
4. Finally, we analyed word frequencies to explore several mechanisms theorized to link nature exposure with mental and cognitive benefits. Negation words such as ‘no’, ‘not’, and ‘don't’ decreased in frequency during visits to urban parks.
5. These results can be used by urban planners and public health officials to better target nature contact recommendations for growing urban populations.

This is not the only study which has shown that having access to, or exposure to nature makes people kinder, happier and stimulates creativity. 

### Audience
The study is conducted for the urban planning department of the city of Brussels. This will allow them to locate areas for new green space development in the city. As such they can more effectively use their available budgets.

The results of the study can potentially benefit all residents of the city of Brussels.

Mental health is an important topic worldwide. With an ever increasing healthcare cost, it is important to explore natural and non-medical factors which can have a substantial benefit on the health of the general public. It is far more important to prevent then to cure.
Taking the above into account, we will attempt to use maps and location data to visualize the balance between greenspace and urban space in different parts of the city. Just as we are building further on a previous data science study based on sentiment analysis, this analysis can then in turn be used later on to link to public health records to map if there is indeed a link between mental health and happiness and relative access to urban greenspace.


References:

https://www.uvm.edu/uvmnews/news/city-parks-lift-mood-much-christmas-twitter-study-shows

https://besjournals.onlinelibrary.wiley.com/doi/full/10.1002/pan3.10045

https://greatergood.berkeley.edu/article/item/how_nature_makes_you_kinder_happier_more_creative



## Data Description
The following data sources can be leveraged to perform this study:

**Foursquare:**

Parks in Brussels:

https://foursquare.com/explore?mode=url&near=Brussels%2C%20Belgium&nearGeoId=72057594040728802&q=Park

Forest in Brussels:

https://foursquare.com/explore?mode=url&near=Brussels%2C%20Belgium&nearGeoId=72057594040728802&q=forest

The above links are a guideline to initiate the study and will be translated in API queries when performing the actual data analysis using the Foursquare Developer API. In case we find that Brussels has a near perfect distribution of parks, we can leverage the average rating values from Foursquare to decide which parks need changes, maintenance or redevelopment.

**Foursquare data items which can potentially be used:**
- Location name for identification
- Latitude and longitude for mapping
- Rating of the location for quality indication
- Amount of tips to estimate popularity

**Open Data Brussels:**
Parks:
https://opendata.brussels.be/explore/dataset/parks/information/

Parks data from the Open Data Brussels platform can perhaps be used during the analysis to see how good the Foursquare coverage is. Based on this the city council could perhaps leverage the use of social internet tools like Foursquare to assure that  residents and tourists can get easy access to info regarding urban green space.

**Open Data Brussels data items which can potentially be used:**
- Location name for identification
- Latitude and longitude for mapping


___
Imports

In [2]:
# uncomment below line if the packages aren't available yet
# !conda install -c anaconda lxml --yes
# !conda install -c conda-forge geopy --yes 

import requests
import pandas as pd
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import unicodedata
import json
from pandas import json_normalize

print("Packages loaded")

Packages loaded


### Loading external data

In [3]:
# Already downloaded
# !wget -q -O 'brussels_park_data.csv' https://opendata.brussels.be/explore/dataset/parks/download/?format=csv
print('Data downloaded!')

Data downloaded!


### Preparing initial dataframes

In [4]:
# Create dataframe from csv
parks_df = pd.read_csv('brussels_park_data.csv', sep=";")
# Split coordinates in lat and long
# create new dataframe containing split lat and long
new = parks_df["coordinates"].str.split(",", n = 1, expand = True)
# convert to float and append new columns to original data frame
# parks_df["name"] = unicodedata.normalize('NFKD', parks_df["name"]).encode('ascii','ignore')
parks_df["latitude"] = new[0].astype(float)
parks_df["longitude"] = new[1].astype(float)
# drop coordinates column
parks_df.drop(columns =["coordinates"], inplace = True) 
# parks_df

### Defining variables

In [5]:
CLIENT_ID = 'HWBYGS0I51FKF5FKQJRVLVJY3SYQLXV32CGCC154ZI2O3O2V' # your Foursquare ID
CLIENT_SECRET = 'ANMCTLFTLEYX0T4TKO50B5XBRUQF3U4BZULVQZPAHIENS52B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HWBYGS0I51FKF5FKQJRVLVJY3SYQLXV32CGCC154ZI2O3O2V
CLIENT_SECRET:ANMCTLFTLEYX0T4TKO50B5XBRUQF3U4BZULVQZPAHIENS52B


### Fetching required data

In [6]:
# Brussels Coordinates
address = 'Brussels, Belgium'
# Use geolocator to get the coordnicates for Brussels, to be used to center our maps later on
geolocator = Nominatim(user_agent="be_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Brussels are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Brussels are 50.8465573, 4.351697.


In [7]:
# Foursquare Park data in Brussels
radius = 5000
#100 is the max limit we can get out of FOurSquare API
# However we need to limit to 50, because we need to fetch the details afterwards for each park separately.
# Foursquare API only allows 50 premium calls per day.
limit = 50
# park = 4bf58dd8d48988d163941735, forest = 52e81612bcbc57f1066b7a23
# Decided to only use parks, forest has no added value
category_id = '4bf58dd8d48988d163941735'


 # create URL to search based on city name for category of parks
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near=Brussels, Belgium&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    limit, 
    category_id)
# url # display URL
#Fetch json data into results var
results = requests.get(url).json()
# results

fs_parks_df = json_normalize(results['response']['groups'][0]['items'])
filtered_columns = ['venue.id', 'venue.name', 'venue.location.lat', 'venue.location.lng']
fs_parks_df = fs_parks_df.loc[:, filtered_columns]
fs_parks_df.columns = ['id', 'name', 'latitude', 'longitude']
# fs_parks_df

print('{} parks were returned by Foursquare.'.format(fs_parks_df.shape[0]))

50 parks were returned by Foursquare.


In [8]:
# Function to get the details for 1 specific venue from Foursquare API
def getVenueDetails(venue_id):
    detail_url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        venue_id,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION)
#     print(detail_url)

    result_detail = requests.get(detail_url).json()
    # return result_detail
    try:
        fs_detail_df = json_normalize(result_detail['response']['venue'])
        detail_filtered_columns = ['id', 'rating', 'stats.tipCount', 'likes.count', 'location.lat', 'location.lng']
        fs_detail_df = fs_detail_df.loc[:, detail_filtered_columns]
        return fs_detail_df
    except:
        print("Exception")
        err_fs_detail_df = pd.DataFrame(columns = ['id', 'rating', 'stats.tipCount', 'likes.count', 'location.lat', 'location.lng'])
        return err_fs_detail_df

In [9]:
#### Commented this section to avoid accidental API calls, considering the rate limit for premium calls is 50 
#### We have all 50 detail records stored in 'foursquare_details.csv', so we cna create dataframe out of this.

# details_df = pd.DataFrame(columns = ['id', 'rating', 'stats.tipCount', 'likes.count', 'location.lat', 'location.lng'])
# for index, row in fs_parks_df.iterrows():
#     fetch_id = row['id']
#     detail = getVenueDetails(fetch_id)
# #     print(detail)
#     details_df = pd.concat([details_df, detail])
# #     print(details_df)
# details_df.to_csv('foursquare_details.csv')
# details_df

##########
# Read Foursquare details in dataframe from csv
details_df = pd.read_csv('foursquare_details.csv')
# We can drop lat and lng
details_df.drop('location.lat', axis=1, inplace=True)
details_df.drop('location.lng', axis=1, inplace=True)
details_df.drop(details_df.columns[0], axis=1, inplace=True)
# details_df

# Join the fetched details to the master data set 
fs_parks_df = fs_parks_df.join(details_df.set_index('id'), on='id')
# Rename columns for readability
fs_parks_df.columns = ['id', 'name', 'latitude', 'longitude', 'rating', 'tipCount', 'likesCount']
# fs_parks_df

# Methodology

## Step 1: Visualize the parks on the map according to Open Brussels Data
We will use folium to generate a map of Brussels with all parks plotted according to the Open Brussels Data platform

In [10]:
# create map of Brussels using latitude and longitude values
map_brussels = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for name, lat, lng in zip(parks_df['name'], parks_df['latitude'], parks_df['longitude']):
    #Use unicodedata normalize, encode and decode to remove accents from names resulting in strange characters when rendered on map
    label = '{}'.format(unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8'))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brussels)  
    
map_brussels

## Step 2: Visualize the parks on the map according to Foursquare

In [11]:
# create map of Brussels using latitude and longitude values
map_brussels_fs = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for name, lat, lng in zip(fs_parks_df['name'], fs_parks_df['latitude'], fs_parks_df['longitude']):
    #Use unicodedata normalize, encode and decode to remove accents from names resulting in strange characters when rendered on map
    label = '{}'.format(unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8'))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brussels_fs)  
    
map_brussels_fs

## We conclude that the Open Brussels Data map is not the best to use as the coverage of Foursquare is more up to date and better.
The Open Brussels data set is outdated as it was created on April 14, 2014. There is no newer version available.  
As such we continue the analysis with the Foursquare data

## Step 3 Clustering of the parks  
We will cluster the parks according to their rating to identify location and amount of parks in relation to certain ratings

In [12]:
# Function to assign corresponding cluster names to the labels
def getClusterName(row):
    if row['Cluster Labels'] == 0:
        return 'Good park'
    elif row['Cluster Labels'] == 1:
        return 'Could be better'
    elif row['Cluster Labels'] == 2:
        return 'Great park'
    elif row['Cluster Labels'] == 3:
        return 'Needs improvement'
    else:
        return 'Other'
    

### Decided to use 4 clusters
We will organize the parks in 4 clusters:
- Great Park
- Good Park
- Could be better
- Needs improvement

In [13]:
# set number of clusters
kclusters = 4

# create new dataframe for clustering., experimentation shows that the only reliable way to get a good output is to drop all columns except for the rating
# This generates clusters according to rating, allowing to identify the quality of the parks
clustering_df = fs_parks_df
clustering_df = clustering_df.drop('name', axis=1)
clustering_df = clustering_df.drop('id', axis=1)
clustering_df = clustering_df.drop('latitude', axis=1)
clustering_df = clustering_df.drop('longitude', axis=1)
clustering_df = clustering_df.drop('tipCount', axis=1)
clustering_df = clustering_df.drop('likesCount', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering_df)

# check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:10] 

# create a new copy to work with, deep=True to make an independent new copy without referencing the old one
fs_mapping_df = fs_parks_df.copy(deep=True)

fs_mapping_df.insert(0, 'Cluster Labels', kmeans.labels_)
# fs_mapping_df.head()


In [14]:
fs_mapping_df['Cluster Name'] = fs_mapping_df.apply(getClusterName, axis=1)

# Results

### Create clustered map

In [15]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]
# print(rainbow)
# rainbow = ['#18AF32', '#1899AF', '#6218AF', '#DD1111']
rainbow = ['#1899AF', '#6218AF', '#18AF32', '#DD1111']

# add markers to the map
markers_colors = []
for lat, lon, cluster, cluster_name, name, rating, tips, likes in zip(fs_mapping_df['latitude'], fs_mapping_df['longitude'], fs_mapping_df['Cluster Labels'], fs_mapping_df['Cluster Name'], fs_mapping_df['name'], fs_mapping_df['rating'], fs_mapping_df['tipCount'], fs_mapping_df['likesCount']):
    label = folium.Popup(str(name) + ' Cluster ' + str(cluster_name) + ' Rating: ' + str(rating) + ' # tips: ' + str(tips) + ' # likes: ' + str(likes), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)

legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 50px; width: 180px; height: 120px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            background-color:white;
                            ">&nbsp;Legend <br>
                              &nbsp;&nbsp;<i class="fa fa-circle fa-lg" style="color:#18AF32"></i>&nbsp; Great park &nbsp; <br>
                              &nbsp;&nbsp;<i class="fa fa-circle fa-lg" style="color:#1899AF"></i>&nbsp; Good park &nbsp; <br>
                              &nbsp;&nbsp;<i class="fa fa-circle fa-lg" style="color:#6218AF"></i>&nbsp; Could be better &nbsp; <br>
                              &nbsp;&nbsp;<i class="fa fa-circle fa-lg" style="color:#DD1111"></i>&nbsp; Needs improvement &nbsp;
                </div>
                ''' 

map_clusters.get_root().html.add_child(folium.Element(legend_html))
map_clusters

## Cluster 1: Great parks - Rating > 9.0

In [16]:
fs_mapping_df.loc[fs_mapping_df['Cluster Labels'] == 2]

,Cluster Labels,id,name,latitude,longitude,rating,tipCount,likesCount,Cluster Name
0,2,4ade0d7ff964a520e86b21e3,Jubelpark / Parc du Cinquantenaire (Jubelpark),50.840654,4.392189,9.3,151,1006,Great park
1,2,4baf1d46f964a520f4eb3be3,Parc Tenboschpark (Tenboschpark),50.820295,4.364380,9.1,18,98,Great park
5,2,4b93c7c6f964a520455134e3,Ter Kamerenbos / Bois de la Cambre (Ter Kamere...,50.806322,4.375214,9.4,74,563,Great park
6,2,4bc19ce92a89ef3bd292f288,Koning Boudewijnpark / Parc Roi Baudouin (Koni...,50.881646,4.321868,9.1,7,52,Great park
8,2,4cc6c1e4b2beb1f768042a4c,Parc de Woluwepark (Woluwepark),50.829686,4.430074,9.5,36,254,Great park
11,2,4c9c451bd3c2b60c9d74b6bc,Park van Abdij Ter Kameren / Parc de l'Abbaye ...,50.817773,4.374128,9.2,5,54,Great park
12,2,4b71b434f964a52060572de3,Josafatpark / Parc Josaphat (Josafatpark),50.861241,4.387321,9.2,30,156,Great park
19,2,4bba10c8cf2fc9b64a15a102,Étangs d'Ixelles / Vijvers van Elsene (Vijvers...,50.824100,4.372925,9.2,14,170,Great park


## Cluster 2: Good parks - Rating between 8.0 and 9.0

In [17]:
fs_mapping_df.loc[fs_mapping_df['Cluster Labels'] == 0]

,Cluster Labels,id,name,latitude,longitude,rating,tipCount,likesCount,Cluster Name
2,0,4ade0d88f964a520176c21e3,Warandepark / Parc de Bruxelles (Warandepark),50.844442,4.363067,8.6,81,531,Good park
3,0,4bdfcad70ee3a5936b6735b0,Parc d'Egmontpark (Egmontpark),50.837804,4.356594,8.8,20,98,Good park
4,0,4c07d5f0271dc9b6ea6f2b9a,Parc Léopold / Leopoldpark (Leopoldpark),50.838603,4.379168,8.7,23,185,Good park
9,0,4bb8c7aab35776b04236c901,Parc Dudenpark (Dudenpark),50.816022,4.330330,8.9,16,81,Good park
10,0,4b93c427f964a520925034e3,Parc du Wolvendaelpark (Wolvendaelpark),50.799073,4.344406,8.8,11,101,Good park
13,0,4baa84cff964a520ea703ae3,Kleine Zavel / Place du Petit Sablon (Kleine Z...,50.839872,4.356024,8.7,12,115,Good park
15,0,4e4d17bd62e1e5f4671f145f,Parc Seny / Senypark (Senypark),50.809460,4.428692,8.9,2,28,Good park
16,0,4c0a549b3c70b713e0f7275b,Parc de Forest / Park van Vorst (Park van Vorst),50.823174,4.336691,8.6,15,98,Good park
17,0,4bc78ac115a7ef3b889a79da,Parc Georges Henri-park (Georges Henri-park),50.845661,4.419336,8.8,25,104,Good park
18,0,4c308ee516adc928b149bf9c,Parc de la Sauvagèrepark (Sauvagèrepark),50.790530,4.365877,8.7,7,34,Good park


## Cluster 3: Could be better - Rating between 7.8 and 8.0

In [18]:
fs_mapping_df.loc[fs_mapping_df['Cluster Labels'] == 1]

,Cluster Labels,id,name,latitude,longitude,rating,tipCount,likesCount,Cluster Name
7,1,4c9b3c2680958cfa407149d4,Maria-Louizasquare / Square Marie-Louise (Mari...,50.847248,4.378141,8.0,9,43,Could be better
14,1,4bd2b7e6caff9521275cd3f0,Square Ambiorixsquare (Ambiorixsquare),50.845947,4.384257,8.2,10,96,Could be better
20,1,4ade0d89f964a5201e6c21e3,Elisabethpark / Parc Élisabeth (Elisabethpark),50.864884,4.326357,8.1,15,74,Could be better
25,1,4f22c8d0e4b06176428ecbc7,Les Jardins du Fleuriste,50.885331,4.345573,7.9,3,17,Could be better
26,1,4bf91d745efe2d7f8cb66b34,Parc Maloupark (Maloupark),50.841218,4.439753,8.1,12,34,Could be better
28,1,4dadb1fe1e72b2444e46df02,Parc de Laeken / Park van Laken (Park van Laken),50.891677,4.349486,8.1,11,91,Could be better
29,1,536bb49e498e2b47e5c80156,Parc Tour & Taxis,50.869448,4.343234,8.0,1,9,Could be better
30,1,4cc6e1e91e596dcb662be167,Kauwberg,50.787584,4.355199,8.0,1,10,Could be better
31,1,4dc51250e4cd169dc642c5fd,Parc Pierre Pauluspark (Pierre Pauluspark),50.829065,4.347695,8.0,4,18,Could be better
32,1,4c6556e8d02b20a1ec2c9d90,Jardin Botanique / Kruidtuin (Kruidtuin),50.854572,4.364791,8.2,44,177,Could be better


## Cluster 4: Needs Improvement - Rating < 7.8

In [19]:
fs_mapping_df.loc[fs_mapping_df['Cluster Labels'] == 3]

,Cluster Labels,id,name,latitude,longitude,rating,tipCount,likesCount,Cluster Name
39,3,4d6f5ecb5b5ca1cd0d6a6b44,Square Armand Steurssquare (Armand Steurssquare),50.853434,4.377987,7.6,1,15,Needs improvement
43,3,4f5cdd19e4b075ad7ad8bed9,Parc Tomberg,50.843068,4.425820,7.5,0,7,Needs improvement
44,3,4b9227ecf964a520c7e933e3,Abdij van Vorst / Abbaye de Forest (Abdij van ...,50.810903,4.317435,7.5,3,18,Needs improvement
45,3,4b52fc16f964a520fc8b27e3,Parc Sobieskipark (Sobieskipark),50.884826,4.346192,7.4,4,11,Needs improvement
46,3,4db151714b22ba776d7f82e1,Parc des Étangs / Vijverspark (Vijverspark),50.825736,4.283016,7.3,4,22,Needs improvement
47,3,4c76db7f923ba143324065e6,Parc Montjoiepark (Montjoiepark),50.809999,4.358759,7.2,1,11,Needs improvement
48,3,4c20804d8b3aa5931bff9e5f,Hof Ter Musschen Park / Parc Hof Ter Musschen ...,50.853769,4.446855,7.1,4,6,Needs improvement
49,3,4c835703d8086dcb19657f52,Parc de Roodebeek / Roodebeekpark (Roodebeekpark),50.849658,4.423155,7.1,8,17,Needs improvement


# Discussion
## Dead spots  
The map visualization allows us to identify dead spots where little to no parks are available. The urban planning department can use this to identify the locations of new parks to be developed.  

## Quality of parks
The clustering of the parks according to the rating allows us to identify parks ranging from "Great Park" to "Needs improvement".  
This data easily allows the urban planning department to identify existing parks that need to be inspected for improvements or potential redevelopment.

## Best parks as an example  
The parks in cluster "Great Parks" can be used for a further study on what makes these parks so great. The final data set also includes how many tips and likes a park has on Foursquare. We can see that the best parks have more tips and likes, so this could be relevant. This allows to inspect what people find good or bad about the park. Eventually this allows to identify how to improve the other parks in the city by applying these same elements.

# Conclusions

**IMPORTANT** Unfortunately the Foursquare API only allows 50 premium calls, these are  required to fetch the details like the park rating. The data is heavily influenced by this rating.  
We are well aware that this study is incomplete, but it could easily be repeated with all data instead of the 50 limit if budget is available to pay for the API.  
As such we will consider these preliminary results, but we know this can be repeated on a full data set with equally relevant outcomes

The clustering approach based on parks ratings gives us good indicators on where to start with planning of new parks and improvement of existing parks.

On a long term this could have a likely benefit to the general well being of the population as several studies have indicated the impact of parks and urban green space on happiness. Overall this would lead to better quality of life, reduced health care costs and potentially a more safe and higher quality environment overall. Something which is important to the capital of Belgium and of the European Union.

## Further steps
As a next step, it would be great to combine additional data to this report to make it more robust. 
- With some budget we can get all relevant data out of the Foursquare API
- It would be interesting to combine police reports on violence and drug use in or around these parks to see the effect on the rating. Maybe the parks are great in facilities, but problematic in terms of safety/crime.
- It would be interesting to take local surveys from the residents about these parks. It is possible that Foursquare is biased towards mostly tourists and likely younger people, but not (elderly) residents.